# Notebook para Pré-Processamento de Dados

## Importação das Bibliotecas

In [1]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

## Load dos dados intermediários

In [2]:
df = pd.read_parquet("../data/interim/sim_2006_2017_eda.parquet")
df

,ASSISTMED,DTOBITO,ESC,ESTCIV,HORAOBITO,IDADE,LOCOCOR,NATURAL,OCUP,RACACOR,SEXO,SUICIDIO
0,NaN,9022006,9.0,4.0,130.0,463.0,3.0,NaN,999993.0,1.0,1,0
1,NaN,26012006,2.0,2.0,1130.0,481.0,3.0,NaN,214305.0,NaN,1,0
2,1.0,19032006,2.0,3.0,1520.0,493.0,3.0,NaN,514105.0,1.0,2,0
3,1.0,21112006,2.0,1.0,1000.0,489.0,3.0,77.0,214305.0,1.0,1,0
4,NaN,16042006,9.0,3.0,2130.0,480.0,3.0,NaN,NaN,1.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3237588,NaN,13042017,NaN,NaN,1840.0,465.0,3.0,835.0,NaN,1.0,1,0
3237589,NaN,19042017,2.0,1.0,937.0,455.0,3.0,829.0,999993.0,4.0,1,0
3237590,1.0,10012017,2.0,3.0,615.0,487.0,2.0,835.0,999993.0,1.0,1,0
3237591,1.0,10082017,2.0,2.0,500.0,486.0,3.0,835.0,999993.0,1.0,1,0


## Feature Selection

* Durante o pré-processamente, identificou-se que algumas colunas que ainda estavam nos dados intermediários não eram tão pertinentes a solução da problemática. São elas as colunas: DTOBITO, HORAOBITO, LOCOCOR, ASSISTMED.

* As features DTOBITO e HORAOBITO representam informações temporais dos óbitos, enquanto LOCOCOR E ASSISTMED são características de localidade e circustância. Contudo, já que o objetivo do projeto e identificar pessoas vulneráveis ao suícidio para conscientizar e evitar fatalidades, essas informações acabam não contribuindo. Por isso optou-se por removê-las da base.

In [3]:
df = df.drop(columns=['DTOBITO', 'HORAOBITO', 'LOCOCOR', 'ASSISTMED'])

## Tratamento de valores nulos e remoção de outliers

* Diversas colunas apresentam valores nulos,simplesmente remover as linhas com essas valores causaria uma grande perda de sinal.
Algumas estratégias devem ser adotadas para preservar a maior qtd. de sinal possível mantendo a maior fidelidade possível com os dados originais.

In [4]:
df.isnull().sum()

ESC         539840
ESTCIV      188999
IDADE          649
NATURAL     957086
OCUP        760072
RACACOR     130117
SEXO             0
SUICIDIO         0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3237593 entries, 0 to 3237592
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ESC       float64
 1   ESTCIV    float64
 2   IDADE     float64
 3   NATURAL   float64
 4   OCUP      float64
 5   RACACOR   float64
 6   SEXO      int64  
 7   SUICIDIO  int64  
dtypes: float64(6), int64(2)
memory usage: 197.6 MB


### Escolaridade

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 1: Nenhuma
  * 2: 1 a 3 anos
  * 3: 4 a 7 anos
  * 4: 8 a 11 anos
  * 5: 12 e mais
  * 9: Ignorado

* Para essa feature, pareceu razoável substituir as linhas com valores nulos pela médias dos outros valores da coluna. Contudo, pelo fato dos casos ignorados serem representados por um valor maior que todos os outros (valor igual a 9), o grupo optou por substituí-lo por um valor mais baixo (0) e depois aplicar a média para os nulos.

In [6]:
# Distribuição dos valores nulos e não-nulos na coluna
# Cabe-se observar a presença de valores iguais a 0, o que não faz parte dos limites encontrados na EDA
# Logo esses valores iguais a 0 são 'outliers' e devem ser descartados
df['ESC'].isnull().sum(), df['ESC'].value_counts()

(539840,
 ESC
 2.0    722256
 3.0    623304
 9.0    409313
 4.0    390218
 1.0    314091
 5.0    173728
 0.0     64843
 Name: count, dtype: int64)

In [7]:
# Remoção dos valores iguais a 0
df = df[df['ESC'] != 0].reset_index(drop=True)

In [8]:
# Substituição do 9 por 0 para não puxar muito a média para cima
df['ESC'] = df['ESC'].replace(9,0)

In [9]:
# Instanciação da classe para substituir os valores nulos
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [10]:
# Usa o imputer para substituir os valores nulos pela média
imputer = imputer.fit(df[['ESC']])
df['ESC'] = imputer.transform(df[['ESC']])

In [11]:
# Distribuição dos valores nulos e não-nulos na coluna
df['ESC'].isnull().sum(), df['ESC'].value_counts()

(0,
 ESC
 2.000000    722256
 3.000000    623304
 2.300886    539840
 0.000000    409313
 4.000000    390218
 1.000000    314091
 5.000000    173728
 Name: count, dtype: int64)

### Estado Civil

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 1: Solteiro
  * 2: Casado
  * 3: Viúvo
  * 4: Separado judicialmente
  * 9: Ignorado

* Para essa feature, adotou-se a estratégia de considerar todos os casos nulos como ignorados (valor igual a 9).

In [12]:
# Distribuição dos valores nulos e não-nulos na coluna
# Cabe-se observar a presença de valores iguais a 5, o que não faz parte dos limites encontrados na EDA
# Logo esses valores iguais a 5 são 'outliers' e devem ser descartados
df['ESTCIV'].isnull().sum(), df['ESTCIV'].value_counts()

(175916,
 ESTCIV
 2.0    1150230
 3.0     838428
 1.0     669040
 4.0     225445
 9.0      81869
 5.0      31822
 Name: count, dtype: int64)

In [13]:
# Remoção dos valores iguais a 5
df = df[df['ESTCIV'] != 5].reset_index(drop=True)

In [14]:
# Considera todos os casos nulos como ignorados
df['ESTCIV'] = df['ESTCIV'].fillna(9)

### Ocupação

* No processo de EDA da base, identificou-se que essa feature representa a ocupação do falecido conforme a Classificação Brasileira de Ocupações (CBO-2002).

* Dito isso, a estratégia adotada foi a de substituir os valores nulos por um valor que já não estivesse presente na coluna e que pudesse caracterizar uma ocupação desconhecida, -1 foi o valor escolhido.

In [15]:
# A coluna não apresenta nenhum registro com ocupação igual a -1, o que torna o valor viável
df[df['OCUP'] == -1]

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,SEXO,SUICIDIO


In [16]:
# Substituição das linhas nulas por -1
df['OCUP'] = df['OCUP'].fillna(-1)

In [17]:
df['OCUP'].isnull().sum()

0

### Raça/Cor

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 1: Branca
  * 2: Preta
  * 3: Amarela
  * 4: Parda
  * 5: Indígena
  * 9: Ignorado

* Para essa feature, adotou-se a estratégia de considerar todos os casos nulos como ignorados (valor igual a 9).

In [18]:
# Distribuição dos valores nulos e não-nulos na coluna
df['RACACOR'].isnull().sum(), df['RACACOR'].value_counts()

(120795,
 RACACOR
 1.0    2290038
 4.0     509722
 2.0     176768
 3.0      42354
 5.0       1246
 9.0          5
 Name: count, dtype: int64)

In [19]:
# Substituição das linhas nulas por 9 (casos ignorados)
df['RACACOR'] = df['RACACOR'].fillna(9)

In [20]:
# Distribuição dos valores nulos e não-nulos na coluna
df['RACACOR'].isnull().sum(), df['RACACOR'].value_counts()

(0,
 RACACOR
 1.0    2290038
 4.0     509722
 2.0     176768
 9.0     120800
 3.0      42354
 5.0       1246
 Name: count, dtype: int64)

### Naturalidade

* No processo de EDA da base, identificou-se que essa feature representa a naturalidade do falecido, conforme a tabela de países. Se for brasileiro, porém, o primeiro dígito contém 8 e os demais o código da UF de naturalidade.

* Dito isso, a estratégia adotada foi a de substituir os valores nulos por um valor que já não estivesse presente na coluna e que pudesse caracterizar uma naturalidade desconhecida, -1 foi o valor escolhido.

In [21]:
# A coluna não apresenta nenhum registro com ocupação igual a -1, o que torna o valor viável
df[df['NATURAL'] == -1]

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,SEXO,SUICIDIO


In [22]:
# Substituição das linhas nulas por -1
df['NATURAL'] = df['NATURAL'].fillna(-1.0)
df['NATURAL'] = df['NATURAL'].astype(np.int64)

In [23]:
# Função para converter as naturalidades para uma notação comum
def padroniza_naturalidade(row):
    nat = row['NATURAL']

    # Se a naturalidade for ignorada (igual a -1) retorna ela sem tratamento
    if nat == -1:
      return -1

    # Caso não seja ignorada, converte pra string
    nat = str(int(nat))

    # Caso seja brasileiro (8 no primeiro dígito) só mantém o código da UF
    if nat[0] == '8' and len(nat) > 1:
        nat = nat[1:]

    # Caso contrário usa a flag -2 para estrangeiro
    else:
        nat = '-2'

    return int(nat)

# Aplica a função na coluna naturalidade
df['NATURAL'] = df.apply(padroniza_naturalidade, axis=1)

In [24]:
# Distribuição dos valores nulos e não-nulos na coluna
df['NATURAL'].isnull().sum()

0

### Idade

* No processo de EDA da base, identificou-se que essa feature representa a idade, composta de dois subcampos. O primeiro, de 1 dígito, indica a unidade da idade. O segundo, de dois dígitos, indica a quantidade de unidades conforme a legenda abaixo:
  * Primeiro dígito igual a 0: Idade ignorada
  * Primeiro dígito igual a 1: Unidade em horas, o segundo subcampo varia de 01 a 23
  * Primeiro dígito igual a 2: Unidade em dias, o segundo subcampo varia de 01 a 29
  * Primeiro dígito igual a 3: Unidade em meses, o segundo subcampo varia de 01 a 11
  * Primeiro dígito igual a 4: Unidade em anos, o segundo subcampo varia de 00 a 99
  * Primeiro dígito igual a 5: Unidade em anos (mais de 100 anos), o segundo subcampo varia de 0 a 99
* Exemplo:
  * 410: 10 anos
  * 457: 57 anos
  * 505: 105 anos

* Para essa feature, primeiramente se descartou idades que não estão em anos, pois não fazem sentido para o objetivo do projeto. Logo após substituiu-se os valores nulos por 0 (ignorados), por fim trocou-se a notação presente pela padrão de idade em anos e depois substituiu-se os valores ignorados pela média da coluna por meio do uso de um imputer.

In [25]:
# Distribuição dos valores nulos e não-nulos na coluna
df['IDADE'].isnull().sum(), df['IDADE'].value_counts()

(649,
 IDADE
 481.0    73112
 480.0    73011
 479.0    72705
 478.0    72391
 482.0    71590
          ...  
 516.0        2
 519.0        1
 520.0        1
 527.0        1
 529.0        1
 Name: count, Length: 249, dtype: int64)

In [26]:
# Troca os valores nulos por 0 (ignorados) e converte para int
df['IDADE'] = df['IDADE'].fillna(0.0)
df['IDADE'] = df['IDADE'].astype(np.int64)

In [27]:
# Só mantem os valores com idade ignorada (igual a 0) e os que estão no range de idade em anos (400 a 599)
df = df[(df['IDADE'] == 0) | ((df['IDADE'] > 400) & (df['IDADE'] <= 599))].reset_index(drop=True)

In [28]:
# Função para converter as idades para notação comum em anos
def padroniza_idade(row):
    idade = row['IDADE']

    # Se a idade for ignorada (igual a 0) retorna ela sem tratamento
    if idade == 0:
      return 0

    # Caso não seja ignorada, converte pra string e calcula o valor correto da idade em anos
    idade = str(int(idade))
    if idade[0] == '4':
        idade = idade[1:]
    elif idade[0] == '5':
        idade = '1' + idade[1:]

    return int(idade)

# Aplica a função na coluna idade
df['IDADE'] = df.apply(padroniza_idade, axis=1)

In [29]:
# Instanciação da classe para substituir os valores nulos
imputer = SimpleImputer(missing_values=0, strategy='mean')

In [30]:
# Usa o imputer para substituir os valores nulos pela média
imputer = imputer.fit(df[['IDADE']])
df['IDADE'] = imputer.transform(df[['IDADE']])

In [31]:
# Distribuição dos valores nulos e não-nulos na coluna
df['IDADE'].isnull().sum(), df['IDADE'].value_counts()

(0,
 IDADE
 81.0     73112
 80.0     73011
 79.0     72705
 78.0     72391
 82.0     71590
          ...  
 116.0        2
 119.0        1
 120.0        1
 127.0        1
 129.0        1
 Name: count, Length: 123, dtype: int64)

### Sexo

* No processo de EDA da base, identificou-se que essa feature é representada pela seguinte legenda:
  * 0: Ignorado
  * 1: Masculino
  * 2: Feminino

* Essa feature não apresentava valores nulos, então somente foi realizada a remoção de outliers.

In [32]:
df['SEXO'].value_counts()

SEXO
1    1682993
2    1362628
0        154
9         24
Name: count, dtype: int64

In [33]:
df = df[df['SEXO'] != 9].reset_index(drop=True)

In [34]:
df['SEXO'].value_counts()

SEXO
1    1682993
2    1362628
0        154
Name: count, dtype: int64

## Tratamento de features categóricas

### One Hot Enconding

* Da forma que a feature SEXO está, o modelo pode acabar por inferir alguma relação ordinal entre os valores 'masculino' (1) e 'feminino' (2), o que não existe na realidade. Diferentemente de uma feature como escolaridade, por exemplo, em que quanto maior o valor numérico maior a escolaridade daquele indivíduo.
* Para contornar isso, foi utilizada uma estratégia de one hot enconding, na qual essa coluna foi subdividida em duas, cada uma representando um dos estados possíveis (masculino ou feminino) podendo assumir os valores "True" ou "False".

In [35]:
# Os valores ignorados (iguais a 0) são muito poucos
df['SEXO'].value_counts()

SEXO
1    1682993
2    1362628
0        154
Name: count, dtype: int64

In [36]:
df = df[df['SEXO'] != 0]

In [37]:
# Criação e concatenação das novas colunas, assim como remoção da coluna antiga para a feature SEXO
one_hot_sexo = pd.get_dummies(df['SEXO'], prefix='SEXO')
df = pd.concat([df, one_hot_sexo], axis=1)
df = df.rename(columns={'SEXO_1' : 'MASCULINO', 'SEXO_2' : 'FEMININO'}).drop(columns=['SEXO'])

In [38]:
# Como o dataframe ficou
df.head(3)

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,SUICIDIO,MASCULINO,FEMININO
0,0.0,4.0,63.0,-1,999993.0,1.0,0,True,False
1,2.0,2.0,81.0,-1,214305.0,9.0,0,True,False
2,2.0,3.0,93.0,-1,514105.0,1.0,0,False,True


## Normalização

* As diferentes features numéricas da base apresentam escalas diferentes. Por exemplo, para a coluna ESC o valor 5 é o maior alto existente, enquanto para IDADE esse valor é 129. Para resolver esse problema, é importante fazer uma normalização dos dados.

In [39]:
#std = StandardScaler()
#columns = ['ESC', 'ESTCIV', 'IDADE', 'OCUP', 'RACACOR']
#df[columns] = std.fit_transform(df[columns])

In [40]:
df = df[['ESC', 'ESTCIV', 'IDADE', 'NATURAL', 'OCUP', 'RACACOR', 'MASCULINO', 'FEMININO', 'SUICIDIO']]
df

,ESC,ESTCIV,IDADE,NATURAL,OCUP,RACACOR,MASCULINO,FEMININO,SUICIDIO
0,0.000000,4.0,63.0,-1,999993.0,1.0,True,False,0
1,2.000000,2.0,81.0,-1,214305.0,9.0,True,False,0
2,2.000000,3.0,93.0,-1,514105.0,1.0,False,True,0
3,2.000000,1.0,89.0,-2,214305.0,1.0,True,False,0
4,0.000000,3.0,80.0,-1,-1.0,1.0,False,True,0
...,...,...,...,...,...,...,...,...,...
3045770,1.000000,1.0,41.0,35,999993.0,9.0,False,True,0
3045771,2.300886,9.0,65.0,35,-1.0,1.0,True,False,0
3045772,2.000000,1.0,55.0,29,999993.0,4.0,True,False,0
3045773,2.000000,3.0,87.0,35,999993.0,1.0,True,False,0


## Salvamento dos dados pré-processados

In [41]:
df.to_parquet("../data/processed/prototype1.parquet")